In [ ]:
import pandas as pd
from glob import glob
import csv
from datetime import datetime

In [3]:
def quote_json_fields(input_path, output_path):
    with open(input_path, "r", newline='', encoding='utf-8') as infile, \
         open(output_path, "w", newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
        
        for row in reader:
            new_row = []
            for cell in row:
                stripped = cell.strip()
                if stripped.startswith("{") and stripped.endswith("}"):
                    # Wrap in quotes if not already quoted
                    cell = f'"{stripped}"' if not (stripped.startswith('"') and stripped.endswith('"')) else cell
                new_row.append(cell)
            writer.writerow(new_row)

In [6]:
quote_json_fields("Data/orders_2025_3.csv", "orders_2025_3_cleaned.csv")

In [ ]:
import os
from glob import glob

#Replace commas inside nested JSON-like {...} structures with semicolons.
def replace_commas_in_json_fields(line: str) -> str:
    output = ""
    buffer = ""
    stack = 0
    inside_json = False

    for ch in line:
        if ch == '{':
            stack += 1
            inside_json = True
            buffer += ch
        elif ch == '}':
            stack -= 1
            buffer += ch
            if stack == 0:
                inside_json = False
                output += buffer.replace(",", ";")
                buffer = ""
        elif inside_json:
            buffer += ch
        else:
            output += ch

    # Append any remaining buffer (in case of malformed JSON)
    output += buffer
    return output

# Folder containing your files
input_folder = "Raw"

# Get all relevant CSV files
all_csvs = glob(os.path.join(input_folder, "*.csv"))

# Output folder for fixed versions
output_folder = "Processed"
os.makedirs(output_folder, exist_ok=True)

for csv_file in all_csvs:
    file_name = os.path.basename(csv_file)
    output_path = os.path.join(output_folder, f"fixed_{file_name}")

    with open(csv_file, "r", encoding="utf-8") as infile, \
         open(output_path, "w", encoding="utf-8") as outfile:
        
        for line in infile:
            fixed_line = replace_commas_in_json_fields(line)
            outfile.write(fixed_line)

    print(f"Processed: {file_name} → {output_path}")



# Directory containing the fixed files
fixed_folder = "Processed"

# Define JSON-like columns that were semicolon-patched
orders_json_cols = ["delivery_info", "subscription_discounts_metadata"]
stores_json_cols = ["platform_fee", "delivery_fee", "pre_sale", "consumer_fee"]
customers_json_cols = []  

def load_and_restore_commas(files, json_columns):
    dfs = []
    for file in files:
        try:
            df = pd.read_csv(file, dtype=str)
            for col in json_columns:
                if col in df.columns:
                    df[col] = df[col].str.replace(";", ",", regex=False)
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    if dfs:
        return pd.concat(dfs, ignore_index=True).drop_duplicates()
    return pd.DataFrame()

# File collection
orders_files = glob(os.path.join(fixed_folder, "fixed_orders_*.csv"))
customers_files = glob(os.path.join(fixed_folder, "fixed_customers_*.csv"))
stores_files = glob(os.path.join(fixed_folder, "fixed_stores_*.csv")) + [os.path.join(fixed_folder, "fixed_stores.csv")]

# Load, repair, and concat
orders_df = load_and_restore_commas(orders_files, orders_json_cols)
customers_df = load_and_restore_commas(customers_files, customers_json_cols)
stores_df = load_and_restore_commas(stores_files, stores_json_cols)


orders_df.loc[
    (
        orders_df['fulfillment_type'].isin(['pickup', 'curbside']) |
        (orders_df['order_type'].isin(['store_credit_reload', 'gift_card', 'subscription_purchase']))
    ) & orders_df['delivery_fee_in_cents'].isna(),
    'delivery_fee_in_cents'
] = 0

# Fill JSON-like fields with empty objects
orders_df['subscription_discounts_metadata'] = orders_df['subscription_discounts_metadata'].fillna('{}')
orders_df['delivery_info'] = orders_df['delivery_info'].fillna('{}')

# Fill notes with empty string
orders_df['notes'] = orders_df['notes'].fillna('')

# Fill scheduled_fulfillment_at with a fallback timestamp (using created_at)
orders_df['scheduled_fulfillment_at'] = orders_df['scheduled_fulfillment_at'].fillna(orders_df['created_at'])

# Clean stores database missing values that are JSON-like fields with empty objects
stores_df['platform_fee'] = stores_df['platform_fee'].fillna('{}')
stores_df['consumer_fee'] = stores_df['consumer_fee'].fillna('{}')

# Save cleaned outputs
orders_df.to_csv("Processed/cleaned_orders.csv", index=False)
customers_df.to_csv("Processed/cleaned_customers.csv", index=False)
stores_df.to_csv("Processed/cleaned_stores.csv", index=False)


import pandas as pd
import sqlite3
import os

# Load Cleaned CSVs
processed_folder = "Processed"
orders_csv = os.path.join(processed_folder, "cleaned_orders.csv")
customers_csv = os.path.join(processed_folder, "cleaned_customers.csv")
stores_csv = os.path.join(processed_folder, "cleaned_stores.csv")

orders_df = pd.read_csv(orders_csv)
customers_df = pd.read_csv(customers_csv)
stores_df = pd.read_csv(stores_csv)

# Create SQLite DB
db_path = os.path.join(processed_folder, "dashboard_chatbot.db")
conn = sqlite3.connect(db_path)

# Load DataFrames into SQLite Tables 
orders_df.to_sql("orders", conn, if_exists="replace", index=False)
customers_df.to_sql("customers", conn, if_exists="replace", index=False)
stores_df.to_sql("stores", conn, if_exists="replace", index=False)

conn.close()

print(f"Loaded all tables into {db_path}")

Processed: stores_2025_4.csv → Processed/fixed_stores_2025_4.csv
Processed: stores_2025_2.csv → Processed/fixed_stores_2025_2.csv
Processed: stores_2025_3.csv → Processed/fixed_stores_2025_3.csv
Processed: stores_2025_1.csv → Processed/fixed_stores_2025_1.csv
Processed: stores_2024_10.csv → Processed/fixed_stores_2024_10.csv
Processed: customers_2025_3.csv → Processed/fixed_customers_2025_3.csv
Processed: customers_2025_2.csv → Processed/fixed_customers_2025_2.csv
Processed: stores_2024_11.csv → Processed/fixed_stores_2024_11.csv
Processed: orders_2025_4.csv → Processed/fixed_orders_2025_4.csv
Processed: stores_2024_12.csv → Processed/fixed_stores_2024_12.csv
Processed: customers_2025_4.csv → Processed/fixed_customers_2025_4.csv
Processed: orders_2025_3.csv → Processed/fixed_orders_2025_3.csv
Processed: stores.csv → Processed/fixed_stores.csv


In [17]:
import sqlite3
conn = sqlite3.connect(db_path)
query = f"""
SELECT
  ROUND(
    SUM(
      CASE
        WHEN DATE(o.created_at) BETWEEN '2025-03-01' AND '2025-03-31'
        THEN o.total_amount_in_cents
        ELSE 0
      END
    ) / 100.0,
    2
  ) AS march_revenue,
  ROUND(
    SUM(
      CASE
        WHEN DATE(o.created_at) BETWEEN '2025-04-01' AND '2025-04-30'
        THEN o.total_amount_in_cents
        ELSE 0
      END
    ) / 100.0,
    2
  ) AS april_revenue
FROM orders AS o
JOIN stores AS s ON o.store_id = s.store_id
WHERE s.name = 'Tikka Shack';

"""

df = pd.read_sql_query(query, conn)

df

,march_revenue,april_revenue
0,78020.08,76174.88


In [120]:
stores_df[stores_df['store_id']=='b93662c5-860a-45dd-9e15-c35611fc9b17']

,store_id,external_store_id,name,active,created_at,updated_at,delivery_fee,platform_fee,consumer_fee,pre_sale
371,b93662c5-860a-45dd-9e15-c35611fc9b17,MLT2B1R9E1RQC,Frank Coffee,True,2024-04-29T21:01:12.103Z,2025-04-15T14:16:58.198Z,"{""fee"":0,""title"":null,""waiver"":0,""enabled_fee""...","{""fee"":25,""type"":""amount"",""enabled_fee"":true,""...","{""pickup"":{""type"":""amount"",""enabled_fee"":false...","{""active"":false}"


In [142]:
import os
from sqlalchemy import create_engine
import pandas as pd
from groq import Groq

SCHEMA_DESCRIPTION = """
(Note: Text in parentheses indicates “column_name (TYPE, brief‐description)”)

Use **SQLite** syntax only. Do not use any MySQL‐specific functions
such as `DATE_SUB`, `CURDATE()`, or `DATE_FORMAT`. Instead, use
`DATE('now', '-X days')`, `DATE('now')`, `strftime(…)`, etc.

Tables:
orders(
    order_id (UUID, primary key),
    store_id (UUID, foreign key → stores.store_id),
    customer_id (UUID, foreign key → customers.customer_id),
    external_location_id (STRING, external system’s location identifier),
    external_order_id (STRING, external system’s order identifier),
    total_amount_in_cents (INTEGER, total order value),
    discount_amount_in_cents (INTEGER, discount applied),
    delivery_fee_in_cents (INTEGER, fee charged for delivery),
    created_at (DATETIME, order creation timestamp),
    updated_at (DATETIME, last update timestamp),
    fulfillment_type (ENUM: “pickup”|“delivery”|“curbside”),
    tip_amount_in_cents (INTEGER, tip given by customer),
    service_fee_in_cents (INTEGER, platform service fee),
    subscription_discounts_metadata (JSON, subscription discount details),
    notes (STRING, free‐text notes on order),
    delivery_info (JSON, delivery details like addresses/times),
    risk_level (INTEGER, fraud risk score: 0 = low, 1 = high),
    order_type (ENUM: “regular_checkout”|“store_credit_reload”|“gift_card”|“subscription_purchase”),
    perdiem_platform_fee_in_cents (INTEGER, PerDiem’s platform fee),
    scheduled_fulfillment_at (DATETIME, scheduled pickup/delivery time)
)
customers(
    customer_id (UUID, primary key),
    store_id (UUID, foreign key → stores.store_id),
    external_customer_id (STRING, external system’s customer ID)
)
stores(
    store_id (UUID, primary key),
    external_store_id (STRING, external system’s store ID),
    name (STRING, store name),
    active (BOOLEAN, store status),
    created_at (DATETIME, store record creation),
    updated_at (DATETIME, store record last update),
    delivery_fee (JSON, store’s base delivery fee settings),
    platform_fee (JSON, store’s platform fee settings),
    consumer_fee (JSON, consumer‐facing fees),
    pre_sale (JSON, whether scheduled orders are allowed)
)
"""

FEW_SHOT_SQL_PROMPT = [
    {
        "role": "system",
        "content": (
            f"{SCHEMA_DESCRIPTION}\n"
            "Convert the user’s natural language request into a valid SQL query for this schema. "
            "**Using SQLite syntax only.** Return only one SQL statement (no extra commentary)."
        )
    },
    # Example 1: Compare week1 vs week2 for a store by name
    {
        "role": "user",
        "content": "Compare the number of orders between March 1–7 and March 8–14, 2025 for store 'Migos Fine Foods'."
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  SUM(CASE WHEN DATE(o.created_at) BETWEEN '2025-03-01' AND '2025-03-07' THEN 1 ELSE 0 END) AS week1_count,\n"
            "  SUM(CASE WHEN DATE(o.created_at) BETWEEN '2025-03-08' AND '2025-03-14' THEN 1 ELSE 0 END) AS week2_count\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods';"
        )
    },
    # Example 2: Total revenue for a store by name in Q1 2025
    {
        "role": "user",
        "content": "Total revenue (in dollars) for 'Migos Fine Foods' from January 1 to March 31, 2025?"
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  ROUND(SUM(o.total_amount_in_cents) / 100.0, 2) AS total_revenue\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods'\n"
            "  AND DATE(o.created_at) BETWEEN '2025-01-01' AND '2025-03-31';"
        )
    },
    # Example 3: Count pickup orders for a specific week
    {
        "role": "user",
        "content": "How many pickup orders did 'Migos Fine Foods' have between March 15 and March 21, 2025?"
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  COUNT(*) AS pickup_orders_week\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods'\n"
            "  AND o.fulfillment_type = 'pickup'\n"
            "  AND DATE(o.created_at) BETWEEN '2025-03-15' AND '2025-03-21';"
        )
    }
]

api_key = os.getenv("GROQ_KEY")
client = Groq(api_key=api_key)

def nl_to_sql(query: str) -> str:
    messages = FEW_SHOT_SQL_PROMPT + [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages,
        temperature=0.0,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()

def fix_sql_with_error(question: str, bad_sql: str, error_msg: str) -> str:
    fix_prompt = [
        {
            "role": "system",
            "content": (
                f"{SCHEMA_DESCRIPTION}\n"
                "One of your previously generated SQL statements failed on SQLite with an error. "
                "Below is the user’s original question, the SQL you provided, and the SQLite error message. "
                "Please correct the SQL to be valid SQLite syntax and satisfy the original request. "
                "Return only the corrected SQL statement (no commentary)."
            )
        },
        {
            "role": "user",
            "content": (
                f"User question: {question}\n"
                f"Bad SQL: {bad_sql}\n"
                f"SQLite error: {error_msg}"
            )
        }
    ]
    #print(fix_prompt)
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=fix_prompt,
        temperature=0.0,
        max_tokens=256
    )
    #print(response.choices[0].message)
    return response.choices[0].message.content.strip()


FEW_SHOT_SUMMARY_PROMPT = [
    {
        "role": "system",
        "content": (
            "You are given:\n"
            "• The user’s original question\n"
            "• The final SQL query that was executed\n"
            "• The resulting table (or an error message)\n\n"
            "Instructions:\n"
            "1. If the result has multiple rows, include a small markdown‐style table showing those rows.  \n"
            "2. Immediately below, draw one or two brief insights with precise numbers.  \n"
            "3. If the insights suggest an opportunity (e.g., a drop in pickup orders, or one store vastly outperforming others), then propose a specific marketing promotion.  \n"
            "4. Otherwise, skip any marketing suggestion.  \n"
            "5. Always use only the rows shown—do not add, infer, or omit values.  \n"
            "6. If there is an error or no rows, reply exactly:\n"
            "   “I’m sorry, I couldn’t retrieve an answer—please rephrase or check the data.”  \n"
            "7. If the single row is 0, reply exactly:\n"
            "   “It seems there are zero matching records—please verify your question.”  \n"
            "8. Otherwise, for a single non‐zero row, answer in one sentence (no table needed) and only add a marketing idea if it follows logically from the insight."
        )
    },
    # ---- FEW-SHOT EXAMPLE 1 ----
    {
        "role": "user",
        "content": "User question: How many months data of orders do I have? and what are those months?"
    },
    {
        "role": "assistant",
        "content": "There are two months with order data: 2025-03 and 2025-04. So the user has data for March 2025 and April 2025."
    }
]



def summarize_result(question: str, sql_query: str, df: pd.DataFrame = None, error_msg: str = None) -> str:
    """
    Summarize the SQL result (or error) in plain language.
    - If error_msg is provided: ask the user to rephrase.
    - If df exists but all values are zero or empty: ask to verify data.
    - Otherwise: summarize the DataFrame’s contents.
    """
    # Build the “results” part of the prompt
    if error_msg:
        result_content = f"Error executing SQL: {error_msg}"
    elif df is None or df.empty:
        result_content = "Result: no rows returned."
    else:
        # Convert small DataFrame to markdown‐style table text
        table_text = df.to_csv(index=False)
        # Check if single value 0
        if df.shape == (1, 1) and str(df.iat[0, 0]) in ("0", "0.0"):
            result_content = "Result: single value 0"
        else:
            result_content = f"Result Table:\n{table_text}"

    messages = FEW_SHOT_SUMMARY_PROMPT + [
    {
        "role": "user",
        "content": (
            f"User question: {question}\n"
            f"SQL Query: {sql_query}\n"
            f"{result_content}"
        )
    }
   ]
    
   #print(messages)

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages,
        temperature=0.0,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    # Natural language question
    user_question = (
        "How did Tikka Shack perform in march 2025 in comparison to april 2025"
    )
    
    generated_sql = nl_to_sql(user_question)
    #print("Initial Generated SQL:")
    #print(generated_sql)

    db_path = "Processed/dashboard_chatbot.db"
    engine = create_engine(f"sqlite:///{db_path}")

    try:
        df_result = pd.read_sql_query(generated_sql, engine)
        #print("\nQuery Result:")
        #print(df_result)
        error_msg = None

    except Exception as e:
        error_msg = str(e)
        #print(f"\nError executing SQL:\n{error_msg}")
        corrected_sql = fix_sql_with_error(user_question, generated_sql, error_msg)
        #print("\nCorrected SQL:")
        #print(corrected_sql)
        try:
            df_result = pd.read_sql_query(corrected_sql, engine)
            #print("\nFixed Query Result:")
            #print(df_result)
            error_msg = None
        except Exception as e2:
            error_msg = str(e)
            df_result = None
            print(f"\nStill failing after correction: {e2}")
            
    
    summary = summarize_result(user_question, generated_sql, df_result, error_msg)
    print("\nAnswer:")
    print(summary)
            






Answer:
| march_orders | april_orders | march_revenue | april_revenue |
| --- | --- | --- | --- |
| 2784 | 2809 | 78020.08 | 76174.88 |

Tikka Shack received 25 more orders in April 2025 compared to March 2025, but the revenue decreased by $1845.20. Consider offering a limited-time discount or promotion to boost sales and revenue in the upcoming months.


In [11]:
import os
from sqlalchemy import create_engine
import pandas as pd
from groq import Groq

# 1) Import ConversationBufferMemory from LangChain
from langchain.memory import ConversationBufferMemory

SCHEMA_DESCRIPTION = """
(Note: Text in parentheses indicates “column_name (TYPE, brief‐description)”)

Use **SQLite** syntax only. Do not use any MySQL‐specific functions
such as `DATE_SUB`, `CURDATE()`, or `DATE_FORMAT`. Instead, use
`DATE('now', '-X days')`, `DATE('now')`, `strftime(…)`, etc.

Tables:
orders(
    order_id (UUID, primary key),
    store_id (UUID, foreign key → stores.store_id),
    customer_id (UUID, foreign key → customers.customer_id),
    external_location_id (STRING, external system’s location identifier),
    external_order_id (STRING, external system’s order identifier),
    total_amount_in_cents (INTEGER, total order value),
    discount_amount_in_cents (INTEGER, discount applied),
    delivery_fee_in_cents (INTEGER, fee charged for delivery),
    created_at (DATETIME, order creation timestamp),
    updated_at (DATETIME, last update timestamp),
    fulfillment_type (ENUM: “pickup”|“delivery”|“curbside”),
    tip_amount_in_cents (INTEGER, tip given by customer),
    service_fee_in_cents (INTEGER, platform service fee),
    subscription_discounts_metadata (JSON, subscription discount details),
    notes (STRING, free‐text notes on order),
    delivery_info (JSON, delivery details like addresses/times),
    risk_level (INTEGER, fraud risk score: 0 = low, 1 = high),
    order_type (ENUM: “regular_checkout”|“store_credit_reload”|“gift_card”|“subscription_purchase”),
    perdiem_platform_fee_in_cents (INTEGER, PerDiem’s platform fee),
    scheduled_fulfillment_at (DATETIME, scheduled pickup/delivery time)
)
customers(
    customer_id (UUID, primary key),
    store_id (UUID, foreign key → stores.store_id),
    external_customer_id (STRING, external system’s customer ID)
)
stores(
    store_id (UUID, primary key),
    external_store_id (STRING, external system’s store ID),
    name (STRING, store name),
    active (BOOLEAN, store status),
    created_at (DATETIME, store record creation),
    updated_at (DATETIME, store record last update),
    delivery_fee (JSON, store’s base delivery fee settings),
    platform_fee (JSON, store’s platform fee settings),
    consumer_fee (JSON, consumer‐facing fees),
    pre_sale (JSON, whether scheduled orders are allowed)
)
"""

FEW_SHOT_SQL_PROMPT = [
    {
        "role": "system",
        "content": (
            f"{SCHEMA_DESCRIPTION}\n"
            "Convert the user’s natural language request into a valid SQL query for this schema. "
            "**Using SQLite syntax only.** Return only one SQL statement (no extra commentary)."
        )
    },
    # Example 1: Compare week1 vs week2 for a store by name
    {
        "role": "user",
        "content": "Compare the number of orders between March 1–7 and March 8–14, 2025 for store 'Migos Fine Foods'."
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  SUM(CASE WHEN DATE(o.created_at) BETWEEN '2025-03-01' AND '2025-03-07' THEN 1 ELSE 0 END) AS week1_count,\n"
            "  SUM(CASE WHEN DATE(o.created_at) BETWEEN '2025-03-08' AND '2025-03-14' THEN 1 ELSE 0 END) AS week2_count\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods';"
        )
    },
    # Example 2: Total revenue for a store by name in Q1 2025
    {
        "role": "user",
        "content": "Total revenue (in dollars) for 'Migos Fine Foods' from January 1 to March 31, 2025?"
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  ROUND(SUM(o.total_amount_in_cents) / 100.0, 2) AS total_revenue\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods'\n"
            "  AND DATE(o.created_at) BETWEEN '2025-01-01' AND '2025-03-31';"
        )
    },
    # Example 3: Count pickup orders for a specific week
    {
        "role": "user",
        "content": "How many pickup orders did 'Migos Fine Foods' have between March 15 and March 21, 2025?"
    },
    {
        "role": "assistant",
        "content": (
            "SELECT\n"
            "  COUNT(*) AS pickup_orders_week\n"
            "FROM orders AS o\n"
            "JOIN stores AS s ON o.store_id = s.store_id\n"
            "WHERE s.name = 'Migos Fine Foods'\n"
            "  AND o.fulfillment_type = 'pickup'\n"
            "  AND DATE(o.created_at) BETWEEN '2025-03-15' AND '2025-03-21';"
        )
    }
]

api_key = os.getenv("GROQ_KEY")
client = Groq(api_key=api_key)

def nl_to_sql(query: str) -> str:
    messages = FEW_SHOT_SQL_PROMPT + [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages,
        temperature=0.0,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()

def fix_sql_with_error(question: str, bad_sql: str, error_msg: str) -> str:
    fix_prompt = [
        {
            "role": "system",
            "content": (
                f"{SCHEMA_DESCRIPTION}\n"
                "One of your previously generated SQL statements failed on SQLite with an error. "
                "Below is the user’s original question, the SQL you provided, and the SQLite error message. "
                "Please correct the SQL to be valid SQLite syntax and satisfy the original request. "
                "Return only the corrected SQL statement (no commentary)."
            )
        },
        {
            "role": "user",
            "content": (
                f"User question: {question}\n"
                f"Bad SQL: {bad_sql}\n"
                f"SQLite error: {error_msg}"
            )
        }
    ]
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=fix_prompt,
        temperature=0.0,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()


FEW_SHOT_SUMMARY_PROMPT = [
    {
        "role": "system",
        "content": (
            "You are given:\n"
            "• The user’s original question\n"
            "• The final SQL query that was executed\n"
            "• The resulting table (or an error message)\n"
            "• The conversation memory so far\n\n"
            "Instructions:\n"
            "1. Primarily use the SQL‐extracted table to draw your insights and answer the question.\n"
            "2. If relevant, incorporate any context from the conversation memory to clarify or enrich your analysis, but do not let memory override the concrete numbers in the table.\n"
            "3. If the result has multiple rows, include a small markdown‐style table showing those rows.\n"
            "4. Immediately below that table, draw one or two brief insights with precise numbers.\n"
            "5. If those insights suggest an opportunity (e.g., a drop in pickup orders, or one store vastly outperforming others), then propose a specific marketing promotion.\n"
            "6. Otherwise, skip any marketing suggestion.\n"
            "7. Always use only the rows shown—do not add, infer, or omit values.\n"
            "8. If there is an error or no rows, first consult the conversation memory to try to answer the question. If you still cannot provide an answer, reply exactly:\n"
            "   “I’m sorry, I couldn’t retrieve an answer—please rephrase or check the data.”\n"
            "9. If the single row is 0, reply exactly:\n"
            "   “It seems there are zero matching records—please verify your question.”\n"
            "10. Otherwise, for a single non‐zero row, answer in one sentence (no table needed) and only add a marketing idea if it follows logically from the insight."
        )
    },
    # ---- FEW-SHOT EXAMPLE 1 ----
    {
        "role": "user",
        "content": "User question: How many months data of orders do I have? and what are those months?"
    },
    {
        "role": "assistant",
        "content": "There are two months with order data: 2025-03 and 2025-04. So the user has data for March 2025 and April 2025."
    }
]

# 2) Instantiate a ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)


def summarize_result(question: str, sql_query: str, df: pd.DataFrame = None, error_msg: str = None) -> str:
    """
    Summarize the SQL result (or error) in plain language,
    injecting memory history as one of the prompt fields, then store the new Q→A in memory.
    """
    # Build the “results” part of the prompt
    if error_msg:
        result_content = f"Error executing SQL: {error_msg}"
    elif df is None or df.empty:
        result_content = "Result: no rows returned."
    else:
        # Convert small DataFrame to markdown‐style table text
        table_text = df.to_csv(index=False)
        # Check if single value 0
        if df.shape == (1, 1) and str(df.iat[0, 0]) in ("0", "0.0"):
            result_content = "Result: single value 0"
        else:
            result_content = f"Result Table:\n{table_text}"

    # 3) Load memory history and include it in the summary prompt
    history_str = memory.load_memory_variables({})["history"]
    
    print(history_str)

    # Construct the messages, injecting the memory as its own user‐role field
    messages = FEW_SHOT_SUMMARY_PROMPT + [
        {
            "role": "user",
            "content": (
                f"Conversation memory so far: {history_str}\n"
                f"User question: {question}\n"
                f"SQL Query: {sql_query}\n"
                f"{result_content}"
            )
        }
    ]

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages,
        temperature=0.0,
        max_tokens=256
    )
    summary = response.choices[0].message.content.strip()

    # 4) Save the new (user_question → summary) pair into memory
    memory.save_context({"user": question}, {"assistant": summary})

    return summary


In [19]:
if __name__ == "__main__":
    # Natural language question
    user_question = "Based on our previous conversations, suggest some strategies"

    generated_sql = nl_to_sql(user_question)

    db_path = "Processed/dashboard_chatbot.db"
    engine = create_engine(f"sqlite:///{db_path}")

    try:
        df_result = pd.read_sql_query(generated_sql, engine)
        error_msg = None

    except Exception as e:
        error_msg = str(e)
        corrected_sql = fix_sql_with_error(user_question, generated_sql, error_msg)
        try:
            df_result = pd.read_sql_query(corrected_sql, engine)
            error_msg = None
        except Exception as e2:
            error_msg = str(e2)
            df_result = None

    summary = summarize_result(user_question, generated_sql, df_result, error_msg)
    print("\nAnswer:")
    print(summary)


[HumanMessage(content='What can I do to improve the sales then?'), AIMessage(content="I'm sorry, I couldn't retrieve an answer—please rephrase or check the data."), HumanMessage(content='How was tikka shack sale during march 2025?'), AIMessage(content="| total_sales |\n| --- |\n| 78020.08 |\n\nTikka Shack's total sales in March 2025 were $78,020.08. Since the user previously asked about improving sales, a potential marketing promotion could be offering a limited-time discount or loyalty reward to customers who make repeat purchases at Tikka Shack."), HumanMessage(content='Compare march 2025 sales of tikka shack to april 2025'), AIMessage(content="| march_sales | april_sales |\n| --- | --- |\n| 2304.81 | 76174.88 |\n\nTikka Shack's sales in March 2025 were $2,304.81, and in April 2025, they were $76,174.88. \n\nA potential marketing promotion could be to continue or expand the successful strategies used in April, as the sales increased significantly."), HumanMessage(content='How did tik

In [23]:
import pandas as pd

# 1. Load cleaned stores to get the store_id for “Coffee Dose”
stores_df = pd.read_csv("Processed/cleaned_stores.csv", dtype=str)
coffee_dose_row = stores_df[stores_df["name"] == "Coffee Dose"]

if coffee_dose_row.empty:
    raise ValueError("No store named 'Coffee Dose' found in cleaned_stores.csv")

coffee_dose_id = coffee_dose_row.iloc[0]["store_id"]

# 2. Load cleaned orders and filter for Coffee Dose
orders_df = pd.read_csv("Processed/cleaned_orders.csv", dtype=str)

# Convert total_amount_in_cents to integer (cents)
orders_df["total_amount_in_cents"] = (
    pd.to_numeric(orders_df["total_amount_in_cents"], errors="coerce")
      .fillna(0)
      .astype(int)
)

# Filter orders for Coffee Dose by store_id
coffee_orders = orders_df[orders_df["store_id"] == coffee_dose_id].copy()

# 3. Parse created_at to datetime and extract YYYY-MM as “month”
coffee_orders["created_at"] = pd.to_datetime(coffee_orders["created_at"], errors="coerce")
coffee_orders["month"] = coffee_orders["created_at"].dt.to_period("M").astype(str)

# 4. Group by month and sum revenue (in cents), then convert to dollars
monthly_revenue_cents = coffee_orders.groupby("month")["total_amount_in_cents"].sum()
monthly_revenue_usd = monthly_revenue_cents / 100.0

# 5. Display results and compute the difference between March 2025 and April 2025
result_df = monthly_revenue_usd.reset_index().rename(
    columns={"total_amount_in_cents": "revenue_usd", 0: "revenue_usd"}
)
result_df.columns = ["month", "revenue_usd"]
print("Coffee Dose revenue by month (in dollars):")
print(result_df.to_string(index=False, formatters={"revenue_usd": "${:,.2f}".format}))

# 6. Verify the change from March 2025 to April 2025
revenue_march = monthly_revenue_usd.get("2025-03", 0.0)
revenue_april = monthly_revenue_usd.get("2025-04", 0.0)
difference = revenue_april - revenue_march

print(f"\nRevenue in March 2025: ${revenue_march:,.2f}")
print(f"Revenue in April 2025: ${revenue_april:,.2f}")
print(f"Difference (April – March): ${difference:,.2f}")


Coffee Dose revenue by month (in dollars):
  month revenue_usd
2025-03 $148,582.58
2025-04 $158,123.60

Revenue in March 2025: $148,582.58
Revenue in April 2025: $158,123.60
Difference (April – March): $9,541.02


/var/folders/lb/q4hhzm1x21ddckmdtyxcjd_r0000gn/T/ipykernel_52603/2557282596.py:27: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  coffee_orders["month"] = coffee_orders["created_at"].dt.to_period("M").astype(str)


In [46]:
orders_df=pd.read_csv("/Users/rohit/Desktop/DataQueryBot/Processed/cleaned_orders.csv")
stores_df=pd.read_csv("/Users/rohit/Desktop/DataQueryBot/Processed/cleaned_stores.csv")
customers_df=pd.read_csv("/Users/rohit/Desktop/DataQueryBot/Processed/cleaned_customers.csv")

In [47]:
import pandas as pd

# (1) Ensure 'created_at' is datetime
orders_df['created_at'] = pd.to_datetime(orders_df['created_at'], errors='coerce')

# (2) Filter to Q1 2025
mask_q1 = (
    (orders_df['created_at'] >= '2025-01-01') &
    (orders_df['created_at'] <  '2025-04-01')
)
q1 = orders_df[mask_q1].copy()

# (3) Compute revenue in dollars
q1['revenue'] = q1['total_amount_in_cents'].astype(int) / 100.0

# (4) Group by fulfillment_type and sum
rev_by_fulfillment = (
    q1
    .groupby('fulfillment_type')['revenue']
    .sum()
    .reset_index(name='total_revenue')
)

# (5) Identify max
max_idx = rev_by_fulfillment['total_revenue'].idxmax()
best_type = rev_by_fulfillment.loc[[max_idx]]
print(rev_by_fulfillment)
print("\nHighest‐revenue fulfillment type:")
print(best_type)


  fulfillment_type  total_revenue
0         curbside           2.89
1         delivery      115489.14
2           pickup     1211278.06

Highest‐revenue fulfillment type:
  fulfillment_type  total_revenue
2           pickup     1211278.06


In [ ]:
stores_df[stores_df['']]

In [100]:
import sqlite3
import pandas as pd

db_path = "Processed/dashboard_chatbot.db"
conn = sqlite3.connect(db_path)

# 1) Get the list of tables
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)
print("Tables in database:", tables['name'].tolist())

# 2) For each table, use PRAGMA table_info(...)
def describe_table(table_name):
    df = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
    df = df.rename(columns={
        'cid': 'index',
        'name': 'column',
        'type': 'datatype',
        'notnull': 'not_null',
        'dflt_value': 'default',
        'pk': 'primary_key'
    })
    return df

print("\n-- orders schema --")
print(describe_table("orders"))

print("\n-- customers schema --")
print(describe_table("customers"))

print("\n-- stores schema --")
print(describe_table("stores"))

conn.close()


Tables in database: ['orders', 'customers', 'stores']

-- orders schema --
    index                           column datatype  not_null default  \
0       0                         order_id     TEXT         0    None   
1       1                         store_id     TEXT         0    None   
2       2                      customer_id     TEXT         0    None   
3       3             external_location_id     TEXT         0    None   
4       4                external_order_id     TEXT         0    None   
5       5            total_amount_in_cents     TEXT         0    None   
6       6         discount_amount_in_cents     TEXT         0    None   
7       7            delivery_fee_in_cents     TEXT         0    None   
8       8                       created_at     TEXT         0    None   
9       9                       updated_at     TEXT         0    None   
10     10                 fulfillment_type     TEXT         0    None   
11     11              tip_amount_in_cents     TE